In [17]:
import json
import requests
from decimal import Decimal, ROUND_HALF_UP
from collections import defaultdict
from datetime import datetime
import time
from dotenv import load_dotenv
from pathlib import Path
import os


#comoo o meu notbebook teste não está na raiz do projeto, preciso subir um nível para encontrar a pasta json
PROJECT_ROOT = Path.cwd ().parent


def analisar_empenho_completo(historico_json, documentos_json, empenho_numero, api_key, delay_requests=1):
    """
    Analisa o empenho calculando o saldo atual e mostrando como foi gasto,
    fazendo consultas adicionais na API para valores precisos.
    
    Args:
        historico_json: JSON com histórico de movimentações do empenho
        documentos_json: JSON com documentos relacionados (pagamentos/liquidações)
        empenho_numero: Número do empenho a ser analisado (ex: "153010152442022NE000013")
        api_key: Chave da API do Portal da Transparência
        delay_requests: Delay entre requests em segundos (padrão: 1)
    
    Returns:
        dict: Análise completa do empenho
    """
    
    # Parse dos JSONs
    if isinstance(historico_json, str):
        historico = json.loads(historico_json)
    else:
        historico = historico_json
        
    if isinstance(documentos_json, str):
        documentos = json.loads(documentos_json)
    else:
        documentos = documentos_json
    
    # Calcular valor atual do empenho baseado no histórico
    valor_empenho_atual = Decimal('0')
    movimentacoes = []
    
    if '1' in historico:
        for mov in historico['1']:
            valor = Decimal(mov['valorTotal'].replace('.', '').replace(',', '.'))
            
            if mov['operacao'] == 'INCLUSAO':
                valor_empenho_atual += valor
                movimentacoes.append({
                    'data': mov['data'],
                    'operacao': 'Inclusão',
                    'valor': valor,
                    'descricao': f"Inclusão inicial de R$ {formatar_valor(valor)}"
                })
            elif mov['operacao'] == 'REFORCO':
                valor_empenho_atual += valor
                movimentacoes.append({
                    'data': mov['data'],
                    'operacao': 'Reforço',
                    'valor': valor,
                    'descricao': f"Reforço de R$ {formatar_valor(valor)}"
                })
            elif mov['operacao'] == 'ANULACAO':
                valor_empenho_atual -= valor
                movimentacoes.append({
                    'data': mov['data'],
                    'operacao': 'Anulação',
                    'valor': -valor,
                    'descricao': f"Anulação de R$ {formatar_valor(valor)}"
                })
    
    # Processar pagamentos com consultas na API
    total_pagamentos = Decimal('0')
    total_liquidacoes = Decimal('0')
    gastos_por_categoria = defaultdict(Decimal)
    gastos_por_favorecido = defaultdict(Decimal)
    detalhes_gastos = []
    erros_consulta = []
    
    print(f"Consultando valores precisos para {empenho_numero}...")
    
    for doc in documentos:
        if doc['fase'] in ['Pagamento', 'Liquidação'] and doc['valor'] != '0,00':
            
            codigo_documento = doc['documento']
            fase = 3 if doc['fase'] == 'Pagamento' else 2
            
            try:
                valor_utilizado = consultar_empenho_impactado(
                    codigo_documento, 
                    fase, 
                    empenho_numero, 
                    api_key
                )
                
                if valor_utilizado > 0:
                    if doc['fase'] == 'Pagamento':
                        total_pagamentos += valor_utilizado
                    else:  # Liquidação
                        total_liquidacoes += valor_utilizado
                    
                    categoria = doc['elementoDespesa']
                    favorecido = doc['favorecido']
                    
                    gastos_por_categoria[categoria] += valor_utilizado
                    gastos_por_favorecido[favorecido] += valor_utilizado
                    
                    detalhes_gastos.append({
                        'data': doc['data'],
                        'documento': doc.get('documentoResumido', doc.get('documento', 'N/A')),
                        'fase': doc['fase'],
                        'valor_documento': Decimal(doc['valor'].replace('.', '').replace(',', '.')),
                        'valor_utilizado': valor_utilizado,
                        'categoria': categoria,
                        'favorecido': favorecido
                    })
                
                time.sleep(delay_requests)
                
            except Exception as e:
                erros_consulta.append({
                    'documento': codigo_documento,
                    'erro': str(e)
                })
                print(f"Erro ao consultar documento {codigo_documento}: {e}")
    
    saldo_empenho = valor_empenho_atual - total_pagamentos
    valor_liquidado_nao_pago = total_liquidacoes - total_pagamentos
    
    gastos_resumo = [f"{categoria}: R$ {formatar_valor(valor)}" for categoria, valor in gastos_por_categoria.items() if valor > 0]
    
    favorecidos_resumo = [f"{favorecido}: R$ {formatar_valor(valor)}" for favorecido, valor in gastos_por_favorecido.items() if favorecido != "Sem informação" and valor > 0]
    
    if gastos_resumo:
        gastos_texto = " | ".join(gastos_resumo)
        frase_final = f"O SALDO DO EMPENHO É R$ {formatar_valor(saldo_empenho)} POIS FOI GASTO EM {gastos_texto}"
    else:
        frase_final = f"O SALDO DO EMPENHO É R$ {formatar_valor(saldo_empenho)} POIS AINDA NÃO FORAM IDENTIFICADOS GASTOS ESPECÍFICOS DESTE EMPENHO"
    
    return {
        'numero_empenho': empenho_numero,
        'valor_original': valor_empenho_atual,
        'total_liquidado': total_liquidacoes,
        'total_pagamentos': total_pagamentos,
        'saldo_atual': saldo_empenho,
        'valor_liquidado_nao_pago': valor_liquidado_nao_pago,
        'percentual_executado': float((total_pagamentos / valor_empenho_atual * 100).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)) if valor_empenho_atual > 0 else 0,
        'movimentacoes': sorted(movimentacoes, key=lambda x: datetime.strptime(x['data'], '%d/%m/%Y')),
        'gastos_por_categoria': dict(gastos_por_categoria),
        'gastos_por_favorecido': dict(gastos_por_favorecido),
        'detalhes_gastos': sorted(detalhes_gastos, key=lambda x: datetime.strptime(x['data'], '%d/%m/%Y')),
        'erros_consulta': erros_consulta,
        'frase_final': frase_final
    }

def consultar_empenho_impactado(codigo_documento, fase, empenho_alvo, api_key):
    """
    Consulta a API para descobrir quanto do documento foi usado no empenho específico.
    
    Args:
        codigo_documento: Código do documento (ex: "153010152442023OB800037")
        fase: Fase da despesa (2=Liquidação, 3=Pagamento)
        empenho_alvo: Número do empenho alvo (ex: "153010152442022NE000013")
        api_key: Chave da API
    
    Returns:
        Decimal: Valor utilizado no empenho específico
    """
    url = "https://api.portaldatransparencia.gov.br/api-de-dados/despesas/empenhos-impactados"
    
    params = {
        'codigoDocumento': codigo_documento,
        'fase': fase,
        'pagina': 1
    }
    
    headers = {
        'accept': '*/*',
        'chave-api-dados': api_key
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        
        # Procurar pelo empenho específico nos resultados
        for item in data:
            if item.get('empenho') == empenho_alvo:
                # Somar todos os valores relacionados ao empenho
                valor_total = Decimal('0')
                
                # Verificar diferentes campos de valor baseado na fase
                if fase == 3:  # Pagamento
                    if 'valorPago' in item and item['valorPago'] != '0,00':
                        valor_total += Decimal(item['valorPago'].replace('.', '').replace(',', '.'))
                    if 'valorRestoPago' in item and item['valorRestoPago'] != '0,00':
                        valor_total += Decimal(item['valorRestoPago'].replace('.', '').replace(',', '.'))
                elif fase == 2:  # Liquidação
                    if 'valorLiquidado' in item and item['valorLiquidado'] != '0,00':
                        valor_total += Decimal(item['valorLiquidado'].replace('.', '').replace(',', '.'))
                
                return valor_total
        
        return Decimal('0')
        
    except requests.RequestException as e:
        raise Exception(f"Erro na requisição HTTP para {codigo_documento}: {e}")
    except json.JSONDecodeError as e:
        raise Exception(f"Erro ao decodificar JSON da resposta para {codigo_documento}: {e}")
    except Exception as e:
        raise Exception(f"Erro inesperado ao processar {codigo_documento}: {e}")

def formatar_valor(valor):
    """Formata valor decimal para padrão brasileiro"""
    if isinstance(valor, str):
        valor = Decimal(valor.replace('.', '').replace(',', '.'))
    
    valor_str = f"{valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    return valor_str

def imprimir_analise_completa(resultado):
    """Imprime análise completa do empenho"""
    print("=" * 80)
    print(f"ANÁLISE COMPLETA DO EMPENHO: {resultado['numero_empenho']}")
    print("=" * 80)
    
    print("\n💰 VALORES:")
    print(f"  Valor do Empenho: R$ {formatar_valor(resultado['valor_original'])}")
    print(f"  Total Liquidado: R$ {formatar_valor(resultado['total_liquidado'])}")
    print(f"  Total Pago: R$ {formatar_valor(resultado['total_pagamentos'])}")
    print(f"  Liquidado não Pago: R$ {formatar_valor(resultado['valor_liquidado_nao_pago'])}")
    print(f"  Saldo Atual: R$ {formatar_valor(resultado['saldo_atual'])}")
    print(f"  Percentual Executado: {resultado['percentual_executado']:.2f}%")
    
    print("\n📋 MOVIMENTAÇÕES DO EMPENHO:")
    for mov in resultado['movimentacoes']:
        sinal = "+" if mov['valor'] > 0 else ""
        print(f"  {mov['data']} - {mov['operacao']}: {sinal}R$ {formatar_valor(abs(mov['valor']))}")
    
    print("\n💸 GASTOS POR CATEGORIA:")
    for categoria, valor in resultado['gastos_por_categoria'].items():
        if valor > 0:
            print(f"  {categoria}: R$ {formatar_valor(valor)}")
    
    print("\n🏢 GASTOS POR FAVORECIDO:")
    for favorecido, valor in resultado['gastos_por_favorecido'].items():
        if favorecido != "Sem informação" and valor > 0:
            print(f"  {favorecido}: R$ {formatar_valor(valor)}")
    
    if resultado['detalhes_gastos']:
        print("\n🔎 DETALHES DOS GASTOS:")
        for gasto in resultado['detalhes_gastos'][-10:]:  # Mostrar últimos 10
            print(f"  {gasto['data']} - {gasto['documento']} ({gasto['fase']})")
            print(f"    Valor do Doc: R$ {formatar_valor(gasto['valor_documento'])}")
            print(f"    Valor Usado: R$ {formatar_valor(gasto['valor_utilizado'])}")
            print(f"    {gasto['categoria']} - {gasto['favorecido']}")
    
    if resultado['erros_consulta']:
        print("\n⚠️ ERROS DE CONSULTA:")
        for erro in resultado['erros_consulta']:
            print(f"  {erro['documento']}: {erro['erro']}")
    
    print("\n🎯 CONCLUSÃO:")
    print(f"  {resultado['frase_final']}")
    print("=" * 80)

if __name__ == "__main__":
    pasta_historicos = PROJECT_ROOT / 'data' / 'raw_details' / 'historicos'
    pasta_pagamentos = PROJECT_ROOT / 'data' / 'raw_details' / 'pagamentos'
    empenho_id = '170116000012023NE000049'
    # Carregue seus JSONs
    try:
        with open(f'{pasta_historicos}/{empenho_id}_historico.json', 'r', encoding='utf-8') as f:
            historico_real = json.load(f)

        with open(f'{pasta_pagamentos}/{empenho_id}_documentos_relacionados.json', 'r', encoding='utf-8') as f:
            documentos_reais = json.load(f)
            
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Certifique-se de que os arquivos JSON estão na mesma pasta do script.")
        print(e)
        historico_real = None
        documentos_reais = None
    
    
    EMPENHO_NUMERO = "170116000012023NE000049"
    caminho_env = Path('.').resolve().parent / '.env'
    load_dotenv(dotenv_path=caminho_env)

    API_KEY = os.getenv('PORTAL_TRANSPARENCIA_API_KEY')

    if historico_real and documentos_reais:
        # Execute a análise completa
        resultado = analisar_empenho_completo(
            historico_real,
            documentos_reais,
            EMPENHO_NUMERO,
            API_KEY,
            delay_requests=1
        )
        
        imprimir_analise_completa(resultado)

Consultando valores precisos para 170116000012023NE000049...
ANÁLISE COMPLETA DO EMPENHO: 170116000012023NE000049

💰 VALORES:
  Valor do Empenho: R$ 398.690,63
  Total Liquidado: R$ 0,00
  Total Pago: R$ 202.679,52
  Liquidado não Pago: R$ -202.679,52
  Saldo Atual: R$ 196.011,11
  Percentual Executado: 50.84%

📋 MOVIMENTAÇÕES DO EMPENHO:
  06/01/2023 - Inclusão: +R$ 10,00
  06/01/2023 - Inclusão: +R$ 10,00
  19/01/2023 - Reforço: +R$ 16.188,79
  19/01/2023 - Reforço: +R$ 16.188,79
  04/04/2023 - Reforço: +R$ 16.178,79
  11/04/2023 - Reforço: +R$ 48.576,37
  11/04/2023 - Reforço: +R$ 48.576,37
  06/06/2023 - Reforço: +R$ 64.755,16
  06/06/2023 - Reforço: +R$ 64.755,16
  26/07/2023 - Reforço: +R$ 80.933,95
  26/07/2023 - Reforço: +R$ 80.933,95
  05/12/2023 - Anulação: R$ 19.208,35
  05/12/2023 - Anulação: R$ 19.208,35

💸 GASTOS POR CATEGORIA:
  Locação de Mão-de-Obra: R$ 202.679,52

🏢 GASTOS POR FAVORECIDO:
  CONSTRUTORA EDIL LTDA: R$ 202.679,52

🔎 DETALHES DOS GASTOS:
  16/11/2023 - 20

In [2]:
import json
import requests
from decimal import Decimal, ROUND_HALF_UP
from collections import defaultdict
from datetime import datetime
import time
from dotenv import load_dotenv
from pathlib import Path
import os


#comoo o meu notbebook teste não está na raiz do projeto, preciso subir um nível para encontrar a pasta json
PROJECT_ROOT = Path.cwd ().parent


def analisar_empenho_completo(historico_json, documentos_json, empenho_numero, api_key, delay_requests=1):
    """
    Analisa o empenho calculando o saldo atual e mostrando como foi gasto,
    fazendo consultas adicionais na API para valores precisos.
    
    Args:
        historico_json: JSON com histórico de movimentações do empenho
        documentos_json: JSON com documentos relacionados (pagamentos/liquidações)
        empenho_numero: Número do empenho a ser analisado (ex: "153010152442022NE000013")
        api_key: Chave da API do Portal da Transparência
        delay_requests: Delay entre requests em segundos (padrão: 1)
    
    Returns:
        dict: Análise completa do empenho
    """
    
    # Parse dos JSONs
    if isinstance(historico_json, str):
        historico = json.loads(historico_json)
    else:
        historico = historico_json
        
    if isinstance(documentos_json, str):
        documentos = json.loads(documentos_json)
    else:
        documentos = documentos_json
    
    # Calcular valor atual do empenho baseado no histórico
    valor_empenho_atual = Decimal('0')
    movimentacoes = []
    
    if '1' in historico:
        for mov in historico['1']:
            valor = Decimal(mov['valorTotal'].replace('.', '').replace(',', '.'))
            
            if mov['operacao'] == 'INCLUSAO':
                valor_empenho_atual += valor
                movimentacoes.append({
                    'data': mov['data'],
                    'operacao': 'Inclusão',
                    'valor': valor,
                    'descricao': f"Inclusão inicial de R$ {formatar_valor(valor)}"
                })
            elif mov['operacao'] == 'REFORCO':
                valor_empenho_atual += valor
                movimentacoes.append({
                    'data': mov['data'],
                    'operacao': 'Reforço',
                    'valor': valor,
                    'descricao': f"Reforço de R$ {formatar_valor(valor)}"
                })
            elif mov['operacao'] == 'ANULACAO':
                valor_empenho_atual -= valor
                movimentacoes.append({
                    'data': mov['data'],
                    'operacao': 'Anulação',
                    'valor': -valor,
                    'descricao': f"Anulação de R$ {formatar_valor(valor)}"
                })
    
    # Processar pagamentos com consultas na API
    total_pagamentos = Decimal('0')
    total_liquidacoes = Decimal('0')
    gastos_por_categoria = defaultdict(Decimal)
    gastos_por_favorecido = defaultdict(Decimal)
    detalhes_gastos = []
    erros_consulta = []
    
    print(f"Consultando valores precisos para {empenho_numero}...")
    
    for doc in documentos:
        if doc['fase'] in ['Pagamento', 'Liquidação'] and doc['valor'] != '0,00':
            
            codigo_documento = doc['documento']
            fase = 3 if doc['fase'] == 'Pagamento' else 2
            
            try:
                valor_utilizado = consultar_empenho_impactado(
                    codigo_documento, 
                    fase, 
                    empenho_numero, 
                    api_key
                )
                
                if valor_utilizado > 0:
                    if doc['fase'] == 'Pagamento':
                        total_pagamentos += valor_utilizado
                    else:  # Liquidação
                        total_liquidacoes += valor_utilizado
                    
                    categoria = doc['elementoDespesa']
                    favorecido = doc['favorecido']
                    
                    gastos_por_categoria[categoria] += valor_utilizado
                    gastos_por_favorecido[favorecido] += valor_utilizado
                    
                    detalhes_gastos.append({
                        'data': doc['data'],
                        'documento': doc.get('documentoResumido', doc.get('documento', 'N/A')),
                        'fase': doc['fase'],
                        'valor_documento': Decimal(doc['valor'].replace('.', '').replace(',', '.')),
                        'valor_utilizado': valor_utilizado,
                        'categoria': categoria,
                        'favorecido': favorecido
                    })
                
                time.sleep(delay_requests)
                
            except Exception as e:
                erros_consulta.append({
                    'documento': codigo_documento,
                    'erro': str(e)
                })
                print(f"Erro ao consultar documento {codigo_documento}: {e}")
    
    saldo_empenho = valor_empenho_atual - total_pagamentos
    valor_liquidado_nao_pago = total_liquidacoes - total_pagamentos
    
    gastos_resumo = [f"{categoria}: R$ {formatar_valor(valor)}" for categoria, valor in gastos_por_categoria.items() if valor > 0]
    
    favorecidos_resumo = [f"{favorecido}: R$ {formatar_valor(valor)}" for favorecido, valor in gastos_por_favorecido.items() if favorecido != "Sem informação" and valor > 0]
    
    if gastos_resumo:
        gastos_texto = " | ".join(gastos_resumo)
        frase_final = f"O SALDO DO EMPENHO É R$ {formatar_valor(saldo_empenho)} POIS FOI GASTO EM {gastos_texto}"
    else:
        frase_final = f"O SALDO DO EMPENHO É R$ {formatar_valor(saldo_empenho)} POIS AINDA NÃO FORAM IDENTIFICADOS GASTOS ESPECÍFICOS DESTE EMPENHO"
    
    return {
        'numero_empenho': empenho_numero,
        'valor_original': valor_empenho_atual,
        'total_liquidado': total_liquidacoes,
        'total_pagamentos': total_pagamentos,
        'saldo_atual': saldo_empenho,
        'valor_liquidado_nao_pago': valor_liquidado_nao_pago,
        'percentual_executado': float((total_pagamentos / valor_empenho_atual * 100).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)) if valor_empenho_atual > 0 else 0,
        'movimentacoes': sorted(movimentacoes, key=lambda x: datetime.strptime(x['data'], '%d/%m/%Y')),
        'gastos_por_categoria': dict(gastos_por_categoria),
        'gastos_por_favorecido': dict(gastos_por_favorecido),
        'detalhes_gastos': sorted(detalhes_gastos, key=lambda x: datetime.strptime(x['data'], '%d/%m/%Y')),
        'erros_consulta': erros_consulta,
        'frase_final': frase_final
    }

def consultar_empenho_impactado(codigo_documento, fase, empenho_alvo, api_key):
    """
    Consulta a API para descobrir quanto do documento foi usado no empenho específico.
    Versão corrigida e simplificada.
    """
    url = "https://api.portaldatransparencia.gov.br/api-de-dados/despesas/empenhos-impactados"
    
    params = {
        'codigoDocumento': codigo_documento,
        'fase': fase,
        'pagina': 1
    }
    
    headers = {
        'accept': '*/*',
        'chave-api-dados': api_key
    }
    
    try:
        print(f"   🔍 Consultando: {codigo_documento} (fase {fase})")
        
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        
        print(f"   📊 Resposta API: {len(data)} empenhos impactados")
        
        # Procurar pelo empenho específico nos resultados
        valor_total = Decimal('0')
        encontrado = False
        
        for item in data:
            if item.get('empenho') == empenho_alvo:
                encontrado = True
                print(f"   ✅ Empenho {empenho_alvo} ENCONTRADO!")
                
                # Verificar campos de valor baseado na fase
                if fase == 3:  # Pagamento
                    campos = ['valorPago', 'valorRestoPago']
                elif fase == 2:  # Liquidação
                    campos = ['valorLiquidado']
                else:
                    campos = []
                
                for campo in campos:
                    valor_str = item.get(campo, '0,00')
                    if valor_str and valor_str != '0,00':
                        valor = Decimal(valor_str.replace('.', '').replace(',', '.'))
                        valor_total += valor
                        print(f"   💰 {campo}: R$ {formatar_valor(valor)}")
                
                break  # Já encontramos, pode sair do loop
        
        if not encontrado:
            print(f"   ❌ Empenho {empenho_alvo} NÃO encontrado nos impactados")
        else:
            print(f"   🎯 Total utilizado: R$ {formatar_valor(valor_total)}")
        
        return valor_total
        
    except requests.RequestException as e:
        error_msg = f"Erro na requisição HTTP: {e}"
        print(f"   ❌ {error_msg}")
        raise Exception(error_msg)
    except json.JSONDecodeError as e:
        error_msg = f"Erro ao decodificar JSON: {e}"
        print(f"   ❌ {error_msg}")
        raise Exception(error_msg)
    except Exception as e:
        error_msg = f"Erro inesperado: {e}"
        print(f"   ❌ {error_msg}")
        raise Exception(error_msg)

def formatar_valor(valor):
    """Formata valor decimal para padrão brasileiro"""
    if isinstance(valor, str):
        valor = Decimal(valor.replace('.', '').replace(',', '.'))
    
    valor_str = f"{valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    return valor_str

def imprimir_analise_completa(resultado):
    """Imprime análise completa do empenho"""
    print("=" * 80)
    print(f"ANÁLISE COMPLETA DO EMPENHO: {resultado['numero_empenho']}")
    print("=" * 80)
    
    print("\n💰 VALORES:")
    print(f"  Valor do Empenho: R$ {formatar_valor(resultado['valor_original'])}")
    print(f"  Total Liquidado: R$ {formatar_valor(resultado['total_liquidado'])}")
    print(f"  Total Pago: R$ {formatar_valor(resultado['total_pagamentos'])}")
    print(f"  Liquidado não Pago: R$ {formatar_valor(resultado['valor_liquidado_nao_pago'])}")
    print(f"  Saldo Atual: R$ {formatar_valor(resultado['saldo_atual'])}")
    print(f"  Percentual Executado: {resultado['percentual_executado']:.2f}%")
    
    print("\n📋 MOVIMENTAÇÕES DO EMPENHO:")
    for mov in resultado['movimentacoes']:
        sinal = "+" if mov['valor'] > 0 else ""
        print(f"  {mov['data']} - {mov['operacao']}: {sinal}R$ {formatar_valor(abs(mov['valor']))}")
    
    print("\n💸 GASTOS POR CATEGORIA:")
    for categoria, valor in resultado['gastos_por_categoria'].items():
        if valor > 0:
            print(f"  {categoria}: R$ {formatar_valor(valor)}")
    
    print("\n🏢 GASTOS POR FAVORECIDO:")
    for favorecido, valor in resultado['gastos_por_favorecido'].items():
        if favorecido != "Sem informação" and valor > 0:
            print(f"  {favorecido}: R$ {formatar_valor(valor)}")
    
    if resultado['detalhes_gastos']:
        print("\n🔎 DETALHES DOS GASTOS:")
        for gasto in resultado['detalhes_gastos'][-10:]:  # Mostrar últimos 10
            print(f"  {gasto['data']} - {gasto['documento']} ({gasto['fase']})")
            print(f"    Valor do Doc: R$ {formatar_valor(gasto['valor_documento'])}")
            print(f"    Valor Usado: R$ {formatar_valor(gasto['valor_utilizado'])}")
            print(f"    {gasto['categoria']} - {gasto['favorecido']}")
    
    if resultado['erros_consulta']:
        print("\n⚠️ ERROS DE CONSULTA:")
        for erro in resultado['erros_consulta']:
            print(f"  {erro['documento']}: {erro['erro']}")
    
    print("\n🎯 CONCLUSÃO:")
    print(f"  {resultado['frase_final']}")
    print("=" * 80)

if __name__ == "__main__":
    pasta_historicos = PROJECT_ROOT / 'data' / 'raw_details' / 'historicos'
    pasta_pagamentos = PROJECT_ROOT / 'data' / 'raw_details' / 'pagamentos'
    empenho_id = '443036442072024NE000072'
    # Carregue seus JSONs
    try:
        with open(f'{pasta_historicos}/{empenho_id}_historico.json', 'r', encoding='utf-8') as f:
            historico_real = json.load(f)

        with open(f'{pasta_pagamentos}/{empenho_id}_documentos_relacionados.json', 'r', encoding='utf-8') as f:
            documentos_reais = json.load(f)
            
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Certifique-se de que os arquivos JSON estão na mesma pasta do script.")
        print(e)
        historico_real = None
        documentos_reais = None
    
    
    EMPENHO_NUMERO = "443036442072024NE000072"
    caminho_env = Path('.').resolve().parent / '.env'
    load_dotenv(dotenv_path=caminho_env)

    API_KEY = os.getenv('PORTAL_TRANSPARENCIA_API_KEY')

    if historico_real and documentos_reais:
        # Execute a análise completa
        resultado = analisar_empenho_completo(
            historico_real,
            documentos_reais,
            EMPENHO_NUMERO,
            API_KEY,
            delay_requests=1
        )
        
        imprimir_analise_completa(resultado)

Consultando valores precisos para 443036442072024NE000072...
   🔍 Consultando: 443033442072025DF802464 (fase 3)
   📊 Resposta API: 1 empenhos impactados
   ✅ Empenho 443036442072024NE000072 ENCONTRADO!
   💰 valorRestoPago: R$ 4.915,69
   🎯 Total utilizado: R$ 4.915,69
   🔍 Consultando: 443033442072025DF801581 (fase 3)
   📊 Resposta API: 1 empenhos impactados
   ✅ Empenho 443036442072024NE000072 ENCONTRADO!
   💰 valorRestoPago: R$ 3.027,80
   🎯 Total utilizado: R$ 3.027,80
   🔍 Consultando: 443036442072024DR800101 (fase 3)
   📊 Resposta API: 1 empenhos impactados
   ✅ Empenho 443036442072024NE000072 ENCONTRADO!
   💰 valorPago: R$ 996,82
   🎯 Total utilizado: R$ 996,82
   🔍 Consultando: 443033442072025DF800844 (fase 3)
   📊 Resposta API: 1 empenhos impactados
   ✅ Empenho 443036442072024NE000072 ENCONTRADO!
   💰 valorRestoPago: R$ 1.820,63
   🎯 Total utilizado: R$ 1.820,63
   🔍 Consultando: 443033442072024DF811083 (fase 3)
   📊 Resposta API: 1 empenhos impactados
   ✅ Empenho 44303644207